<a href="https://colab.research.google.com/github/iamujjwal98/ml_projects/blob/master/review_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np
import tensorflow_hub as hub
from tensorflow import keras
import tensorflow_datasets as tfds
print(tf.__version__)
print(hub.__version__)
print(keras.__version__)
print("Eager mode: ", tf.executing_eagerly())
if tf.config.experimental.list_physical_devices("GPU"):
  print("yes GPU is being used")
else:
  print("no GPU")  

In [5]:
train_data,validation_data,test_data=tfds.load(name="imdb_reviews",
          split=('train[:60%]','train[60%:]','test'),
          as_supervised=True)


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteM93B4R/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteM93B4R/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteM93B4R/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [0]:
train_example_batch,train_label_batch=next(iter(train_data.batch(10)))

In [8]:
train_example_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [10]:
train_label_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [0]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer=hub.KerasLayer(embedding, 
                output_shape=[20],
                input_shape=[],dtype=tf.string,trainable=True
                )


In [0]:
embbed=hub.load("https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1")
vectors=embbed(["ujjwal is a good guy"])
print(vectors)

In [0]:
hub_layer(train_example_batch[:3])

In [0]:
model=tf.keras.models.Sequential()
model.add(hub_layer)#size of input will 20 which is the output of hub_layer
model.add(tf.keras.layers.Dense(16,activation='relu'))
model.add(tf.keras.layers.Dense(1))

In [0]:
model.summary()

In [0]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [0]:
model.fit(train_data.shuffle(10000).batch(512),
          epochs=20,
          validation_data=validation_data.batch(512),
          verbose=1          
          )

In [28]:
result=model.evaluate(test_data.batch(512),verbose=2)
for name,value in zip(model.metrics_names,result):
  print("%s: %.3f" % (name, value))

49/49 - 3s - loss: 0.3218 - accuracy: 0.8544
loss: 0.322
accuracy: 0.854
